In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import sivtempfit as stf
import sivtempfit.io as io
import sivtempfit.inferMC as mc
import emcee
import sivtempfit.model as model
import numpy as np
import scipy.stats as stats
import seaborn as sns
import pandas as pd

In [2]:
path_to_json = io.get_example_data_file_path("simulated_spectrum.json")
test_spectrum = io.load_Spectrum(path_to_json)

In [3]:
mc.generate_sample_ball(test_spectrum, 70, debug = True)

((33.6339542069,
  33.6339542069,
  669,
  70,
  2,
  0.02,
  0.21816910569000003,
  3.4907056910400005,
  10),
 (26.907163365520002,
  26.907163365520002,
  3,
  0.5,
  2,
  0.05,
  0.21816910569000003,
  1.7453528455200003,
  10))

Ok, let's debug it:

In [4]:
x = test_spectrum.data.values.T[1]
y = test_spectrum.data.values.T[0]
amp1, amp2, C0, center2, width1, width2, light_background, \
            ccd_background, ccd_stdev = [  4.95e+01, 3.50e+01, 6.69e+02, 6.90927784e+01, 
                                         2.97509484e+00, 1.59711767e-02, -1.51653532e-01, 
                                         3.65541083e+00, 2.46400782e+00]
test_model = model.two_peak_log_likelihood(x, y, amp1, amp2, 0, 0, C0,
                    center2, width1, width2, light_background, ccd_background,
                    ccd_stdev, test_norm = False, debug = True)

In [ ]:
np.min(test_model[-1])

In [ ]:
stats.poisson.pmf(1, -10)

In [ ]:
model.two_peak_log_likelihood(x, y, amp1, amp2, 0, 0, C0,
                    center2, width1, width2, light_background, ccd_background,
                    ccd_stdev, test_norm = True)

Ok, I made some improvements, so it should work now. Let's try running the sampler:

In [ ]:
%time test_sampler = mc.mc_likelihood_sampler(test_spectrum, 70, nwalkers = 18, nsteps = 10)

This is insanely slow: ten steps takes about four minutes on my relatively beefy desktop. Now that we have a sampler object, however trivial, we can check out the trajectories.

In [ ]:
fig, (ax_amp1, ax_amp2, ax_c0, ax_center2, ax_width1, ax_width2, ax_lb, ax_ccdb, ax_ccds) = plt.subplots(9)
ax_amp1.set(ylabel='A1')
ax_amp2.set(ylabel='A2')
ax_c0.set(ylabel='CO')
ax_center2.set(ylabel='C2')
ax_width1.set(ylabel='W1')
ax_width2.set(ylabel='W2')
ax_lb.set(ylabel='LB')
ax_ccdb.set(ylabel='CB')
ax_ccds.set(ylabel='Cs')
for i in range(9):
    sns.tsplot(test_sampler.chain[i,:,0], ax=ax_amp1)
    sns.tsplot(test_sampler.chain[i,:,1], ax=ax_amp2)
    sns.tsplot(test_sampler.chain[i,:,2], ax=ax_c0)
    sns.tsplot(test_sampler.chain[i,:,3], ax=ax_center2)
    sns.tsplot(test_sampler.chain[i,:,4], ax=ax_width1)
    sns.tsplot(test_sampler.chain[i,:,5], ax=ax_width2)
    sns.tsplot(test_sampler.chain[i,:,6], ax=ax_lb)
    sns.tsplot(test_sampler.chain[i,:,7], ax=ax_ccdb)
    sns.tsplot(test_sampler.chain[i,:,8], ax=ax_ccds)

In [ ]:
test_sampler.chain?

In [ ]:
samples = test_sampler.chain
# reshape the samples into a 1D array where the colums are m and b
traces = samples.reshape(-1, 9).T
# create a pandas DataFrame with labels.  This will come in handy 
# in a moment, when we start using seaborn to plot our results 
# (among other things, it saves us the trouble of typing in labels
# for our plots)
parameter_samples = pd.DataFrame({'C0': traces[2]-739, 'C2': traces[3], 'w1': traces[4]})
sns.pairplot(parameter_samples, markers='.')

In [ ]:
print(test_spectrum.metadata['Description'])

In [ ]:
traces[2]-739